# Orthophoto Download and Processing
 
**Objective:** Download SITG orthophotos, generate tiles and visualizations.
 
**Workflow:**
1. URL discovery and validation for orthophoto archives
2. Parallel download management
3. Extraction and processing
4. Tile generation and coverage analysis

## Imports

In [ ]:
import aiohttp
import asyncio
from datetime import datetime
from loguru import logger
from tqdm.notebook import tqdm
from pathlib import Path
from asyncio import Queue
from collections import Counter
import glob
import time
import os
import rasterio
import geopandas as gpd
from shapely.geometry import box
import warnings
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Suppress non-critical warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

## Configuration

In [ ]:
UPDATE_URL = False
OUTPUT_URL_CSV_PATH = "data/notebook_03/url"
OUTPUT_LOGS_PATH = "data/notebook_03/logs"

GEOTIFF_PATH = "data/SITG/ortho2019"
GEOTIFF_ZIP_PATH = "/data/denis.iglesias/TM_rooftops/SITG/ortho_2019_zip_2024-11-10"

OUTPUT_QUADRILLAGE_PARQUET_PATH = "data/notebook_03/parquet/03_quadrillage.parquet"
VIZ_OUTPUT_PATH = "data/notebook_03/graphics"


## URL Discovery for GeoTIFF Archives

In [ ]:
# Initialize logging directory
log_dir = Path(OUTPUT_LOGS_PATH)
log_dir.mkdir(exist_ok=True)

# Configure structured logging
logger.remove()
logger.add(
    log_dir / "url_checker_{time}.log",
    rotation="1 day",
    retention="7 days",
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
    level="INFO",
)

# Add console output for real-time monitoring
logger.add(lambda msg: tqdm.write(msg), level="INFO", format="{message}")

async def check_url(session, number, sem, queue):
    """
    Check if a URL exists for a given orthophoto tile number.
    
    Uses HEAD requests for efficiency - only checks if resource exists
    without downloading the actual content.
    
    Parameters:
        session: aiohttp session for connection pooling
        number: Tile number to check
        sem: Semaphore for concurrency control
        queue: Queue for collecting valid URLs
    
    Returns:
        bool: True if URL is valid (200 status)
    """
    base_url = "https://ge.ch/sitg/geodata/SITG/TELECHARGEMENT/ORTHO_2019"
    filename = f"{number}.tif.zip"
    url = f"{base_url}/{filename}"

    async with sem:  # Respect concurrency limits
        try:
            async with session.head(
                url, timeout=aiohttp.ClientTimeout(total=3)
            ) as response:
                if response.status == 200:
                    await queue.put(url)
                    return True
                return False
        except Exception:
            return False

async def progress_reporter(queue, total_urls):
    """
    Report progress statistics during URL discovery.
    
    Provides real-time feedback on discovery rate and estimated completion.
    """
    valid_urls = []
    start_time = time.time()
    counter = Counter()

    while True:
        url = await queue.get()
        if url == "DONE":
            break

        valid_urls.append(url)
        counter["found"] += 1

        # Report progress at regular intervals
        if counter["found"] % 10 == 0:
            elapsed_time = time.time() - start_time
            speed = counter["found"] / elapsed_time if elapsed_time > 0 else 0
            logger.info(
                f"Found {counter['found']} URLs | "
                f"Speed: {speed:.2f} URLs/s | "
                f"Progress: {(counter['found']/total_urls*100):.2f}%"
            )

    return valid_urls

async def main():
    """
    Main async function for URL discovery.
    
    Scans a range of potential tile numbers to find valid orthophoto URLs.
    Uses high concurrency for efficiency while respecting server limits.
    """
    # Define search range based on known tile numbering pattern
    start_number = 24_800_000
    end_number = 25_100_000
    total_urls = end_number - start_number + 1

    # Performance tuning parameters
    max_concurrent = 1000  # Concurrent connections limit
    chunk_size = 1500      # URLs to process per batch

    sem = asyncio.Semaphore(max_concurrent)
    queue = Queue()
    valid_urls = []

    # Start progress monitoring
    reporter_task = asyncio.create_task(progress_reporter(queue, total_urls))

    async with aiohttp.ClientSession() as session:
        # Process URLs in chunks to avoid overwhelming memory
        for chunk_start in tqdm(range(start_number, end_number + 1, chunk_size)):
            chunk_end = min(chunk_start + chunk_size, end_number + 1)
            chunk_numbers = range(chunk_start, chunk_end)

            tasks = [check_url(session, num, sem, queue) for num in chunk_numbers]
            await asyncio.gather(*tasks)

    # Signal completion and collect results
    await queue.put("DONE")
    valid_urls = await reporter_task

    # Save results with timestamp for tracking
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"{OUTPUT_URL_CSV_PATH}/valid_urls_{timestamp}.txt"

    with open(output_file, "w") as f:
        for url in valid_urls:
            f.write(f"{url}\n")

    logger.info(f"Discovery complete: {len(valid_urls)} valid URLs found")
    logger.info(f"Results saved to: {output_file}")

if UPDATE_URL:
    # Execute URL discovery
    await main()

### Verify Downloaded Files

In [ ]:
def consolidate_url_lists():
    """
    Combine multiple URL discovery runs and check download status.
    
    Handles duplicate URLs across multiple discovery sessions and
    identifies which files still need to be downloaded.
    """
    import glob

    # Gather all URL lists except combined ones
    txt_files = glob.glob(f"{OUTPUT_URL_CSV_PATH}/*.txt")
    txt_files = [f for f in txt_files if "combined" not in f]

    # Consolidate unique URLs
    valid_urls = set()
    for txt_file in txt_files:
        with open(txt_file, "r") as f:
            urls = f.readlines()
            valid_urls.update(urls)

    # Save consolidated list
    output_file = f"{OUTPUT_URL_CSV_PATH}/00_valid_urls_combined.txt"
    print(f"Saving {len(valid_urls)} unique URLs to {output_file}")

    with open(output_file, "w") as f:
        f.writelines(valid_urls)

    return valid_urls

def check_download_status(valid_urls):
    """
    Compare discovered URLs against downloaded files.
    
    Identifies which files have been successfully downloaded and
    which are still pending.
    """
    # Extract filenames from URLs
    valid_filenames = [url.split("/")[-1].strip() for url in valid_urls]
    filename_to_url = {url.split("/")[-1].strip(): url.strip() for url in valid_urls}

    # Check existing downloads
    zip_files = os.listdir(GEOTIFF_ZIP_PATH)
    downloaded_files = set(valid_filenames).intersection(zip_files)

    print(f"Download status:")
    print(f"  Already downloaded: {len(downloaded_files)} files")
    
    # Identify missing files
    missing_files = set(valid_filenames) - downloaded_files
    print(f"  Still needed: {len(missing_files)} files")

    if missing_files:
        print("\nMissing files URLs:")
        for filename in missing_files:
            print(filename_to_url[filename])

    return downloaded_files, missing_files

# Execute consolidation and status check
valid_urls = consolidate_url_lists()
downloaded, missing = check_download_status(valid_urls)

## Generate Orthophoto Tile Index
### Tile Index Creation

In [ ]:
def read_tfw(tif_path):
    """
    Read TFW (World File) associated with a GeoTIFF.
    
    TFW files contain georeferencing information in a simple text format:
    - Line 1: Pixel X size
    - Line 2: Rotation about Y axis
    - Line 3: Rotation about X axis
    - Line 4: Pixel Y size (usually negative)
    - Line 5: X coordinate of upper left pixel center
    - Line 6: Y coordinate of upper left pixel center
    
    Parameters:
        tif_path (Path): Path to the GeoTIFF file
        
    Returns:
        dict: TFW parameters if found, None otherwise
    """
    tif_path = Path(tif_path)
    
    # Check common TFW file extensions
    tfw_paths = [
        tif_path.with_suffix(".tfw"),
        tif_path.with_suffix(".tifw"),
        Path(str(tif_path.with_suffix("")) + ".tfw"),
    ]

    for tfw_path in tfw_paths:
        if tfw_path.exists():
            try:
                with open(tfw_path, "r") as f:
                    lines = [float(line.strip()) for line in f.readlines()]

                if len(lines) != 6:
                    print(
                        f"Warning: Invalid TFW format in {tfw_path} "
                        f"(expected 6 lines, found {len(lines)})"
                    )
                    return None

                return {
                    "x_scale": lines[0],
                    "y_rotation": lines[1],
                    "x_rotation": lines[2],
                    "y_scale": lines[3],
                    "x_origin": lines[4],
                    "y_origin": lines[5],
                    "tfw_path": str(tfw_path),
                }
            except Exception as e:
                print(f"Warning: Error reading TFW file {tfw_path}: {str(e)}")
                return None

    return None

def process_single_geotiff(tif_path):
    """
    Extract comprehensive metadata from a single GeoTIFF file.
    
    Processes both embedded GeoTIFF metadata and external TFW files,
    creating a unified spatial footprint for the image.
    
    Parameters:
        tif_path (Path): Path to GeoTIFF file
        
    Returns:
        dict: Metadata including geometry, CRS, and file properties
    """
    try:
        tif_path = Path(tif_path)
        result = {
            "file_path": str(tif_path),
            "file_name": tif_path.name,
            "size_mb": os.path.getsize(tif_path) / (1024 * 1024),
            "errors": [],
        }

        # Prioritize TFW data as it's often more reliable for Swiss data
        tfw_data = read_tfw(tif_path)
        if tfw_data:
            result.update(
                {
                    "has_tfw": True,
                    "tfw_path": tfw_data["tfw_path"],
                    "tfw_x_scale": tfw_data["x_scale"],
                    "tfw_y_scale": tfw_data["y_scale"],
                    "tfw_x_rotation": tfw_data["x_rotation"],
                    "tfw_y_rotation": tfw_data["y_rotation"],
                    "tfw_x_origin": tfw_data["x_origin"],
                    "tfw_y_origin": tfw_data["y_origin"],
                }
            )

            # Infer CRS from coordinate ranges (Swiss specific)
            if (
                2485000 <= tfw_data["x_origin"] <= 2834000
                and 1075000 <= tfw_data["y_origin"] <= 1299000
            ):
                result["suggested_crs"] = "EPSG:2056"  # CH1903+/LV95
            elif (
                485000 <= tfw_data["x_origin"] <= 834000
                and 75000 <= tfw_data["y_origin"] <= 299000
            ):
                result["suggested_crs"] = "EPSG:21781"  # CH1903/LV03
            else:
                result["suggested_crs"] = None
        else:
            result["has_tfw"] = False
            result["suggested_crs"] = None

        # Read embedded GeoTIFF metadata
        with rasterio.open(tif_path) as src:
            # Basic raster properties
            result.update(
                {
                    "width": src.width,
                    "height": src.height,
                    "band_count": src.count,
                    "data_type": str(src.profile.get("dtype")),
                    "resolution_x": src.res[0] if src.res else None,
                    "resolution_y": src.res[1] if src.res else None,
                    "has_crs": src.crs is not None,
                }
            )

            # Handle CRS detection
            if src.crs:
                result["original_crs"] = src.crs.to_string()
            else:
                result["original_crs"] = result.get("suggested_crs")
                if result["original_crs"]:
                    result["errors"].append(
                        f"No CRS in GeoTIFF, using suggested CRS from TFW: {result['original_crs']}"
                    )
                else:
                    result["errors"].append(
                        "No CRS found and cannot determine from coordinates"
                    )

            # Extract transform information
            transform = src.transform
            if transform and transform.is_identity is False:
                result.update(
                    {
                        "geotiff_x_scale": transform.a,
                        "geotiff_y_scale": transform.e,
                        "geotiff_x_rotation": transform.b,
                        "geotiff_y_rotation": transform.d,
                        "geotiff_x_origin": transform.c,
                        "geotiff_y_origin": transform.f,
                    }
                )

            # Create spatial footprint - prefer TFW if available
            if result.get("has_tfw"):
                # Calculate bounds from TFW parameters
                left = result["tfw_x_origin"]
                top = result["tfw_y_origin"]
                right = left + (src.width * result["tfw_x_scale"])
                bottom = top + (src.height * result["tfw_y_scale"])
                footprint = box(left, bottom, right, top)
                crs = result.get("suggested_crs", "EPSG:2056")
            else:
                # Use GeoTIFF bounds
                bounds = src.bounds
                footprint = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
                crs = src.crs or "EPSG:2056"

            # Create temporary GeoDataFrame for CRS transformation
            temp_gdf = gpd.GeoDataFrame(geometry=[footprint], crs=crs)

            # Ensure consistent CRS (Swiss LV95)
            if temp_gdf.crs != "EPSG:2056":
                try:
                    temp_gdf = temp_gdf.to_crs("EPSG:2056")
                except Exception as e:
                    result["errors"].append(
                        f"CRS transformation failed: {str(e)}"
                    )
                    return None

            # Validate geometry
            if not temp_gdf.geometry[0].is_valid:
                temp_gdf.geometry = temp_gdf.geometry.buffer(0)
                result["errors"].append("Applied geometry correction")

            if np.any(np.isinf(temp_gdf.geometry[0].bounds)):
                result["errors"].append("Invalid bounds detected")
                return None

            result["geometry"] = temp_gdf.geometry[0]

        return result

    except Exception as e:
        print(f"Error processing {tif_path}: {str(e)}")
        return None

def create_geotiff_index(input_folder, output_parquet, recursive=True):
    """
    Create a comprehensive spatial index of all GeoTIFF files.
    
    Generates a GeoParquet file containing footprints and metadata
    for efficient spatial queries and coverage analysis.
    
    Parameters:
        input_folder (str): Directory containing GeoTIFF files
        output_parquet (str): Output path for GeoParquet file
        recursive (bool): Search subdirectories
        
    Returns:
        GeoDataFrame: Spatial index of all GeoTIFF files
    """
    input_path = Path(input_folder)
    if not input_path.exists():
        raise ValueError(f"Input folder does not exist: {input_folder}")

    # Locate all GeoTIFF files
    if recursive:
        tif_files = list(input_path.rglob("*.tif")) + list(input_path.rglob("*.tiff"))
    else:
        tif_files = list(input_path.glob("*.tif")) + list(input_path.glob("*.tiff"))

    if not tif_files:
        raise ValueError(f"No GeoTIFF files found in {input_folder}")

    print(f"Found {len(tif_files)} GeoTIFF files")

    # Process files with progress tracking
    results = []
    for tif_path in tqdm(tif_files, desc="Processing GeoTIFFs"):
        try:
            result = process_single_geotiff(tif_path)
            if result:
                results.append(result)
        except Exception as e:
            print(f"Skipping {tif_path}: {str(e)}")
            continue

    if not results:
        raise ValueError("No GeoTIFF files could be processed successfully")

    # Create spatial dataframe
    gdf = gpd.GeoDataFrame(results, crs="EPSG:2056")

    # Add relative paths for portability
    gdf["relative_path"] = gdf["file_path"].apply(
        lambda x: str(Path(x).relative_to(input_path))
    )

    # Ensure geometry integrity
    gdf.set_geometry('geometry', inplace=True)
    
    if gdf.crs is None:
        gdf.set_crs(epsg=2056, inplace=True)

    print("Validating geometries...")
    gdf['geometry'] = gdf['geometry'].buffer(0)

    # Convert error lists to strings for Parquet compatibility
    gdf['errors'] = gdf['errors'].apply(lambda x: str(x))

    # Save in multiple formats for flexibility
    gdf.to_parquet(output_parquet)
    gdf.to_file(output_parquet.replace(".parquet", ".gpkg"), driver="GPKG")
    print("Saved spatial index to:")
    print(f"  - {output_parquet}")
    print(f"  - {output_parquet.replace('.parquet', '.gpkg')}")

    return gdf

def analyze_tfw_geotiff(gdf):
    """
    Analyze consistency between TFW and GeoTIFF metadata.
    
    Identifies discrepancies that might indicate data quality issues.
    """
    print("\n=== METADATA ANALYSIS ===")
    print(f"Total files processed: {len(gdf)}")
    print(f"Files with TFW: {gdf['has_tfw'].sum()}")
    print(f"Files with embedded CRS: {gdf['has_crs'].sum()}")

    # CRS distribution
    print("\nCRS Distribution:")
    print(gdf["original_crs"].value_counts())

    # Suggested CRS from coordinates
    print("\nInferred CRS from coordinates:")
    print(gdf["suggested_crs"].value_counts())

    # Resolution analysis for TFW files
    tfw_files = gdf[gdf["has_tfw"]]
    if not tfw_files.empty:
        print("\nTFW Resolution Statistics:")
        print(f"X resolution: {tfw_files['tfw_x_scale'].min():.3f} to {tfw_files['tfw_x_scale'].max():.3f}")
        print(f"Y resolution: {tfw_files['tfw_y_scale'].min():.3f} to {tfw_files['tfw_y_scale'].max():.3f}")

        # Check for rotated images
        rotated = tfw_files[
            (tfw_files["tfw_x_rotation"] != 0) | (tfw_files["tfw_y_rotation"] != 0)
        ]
        if not rotated.empty:
            print(f"\nFound {len(rotated)} rotated images")

    # Compare TFW vs GeoTIFF parameters
    mask = gdf["has_tfw"] & gdf["has_crs"]
    if mask.any():
        comparison = gdf[mask].copy()
        print("\nTFW vs GeoTIFF Parameter Comparison:")
        
        matches = 0
        total = len(comparison)

        for _, row in comparison.iterrows():
            if all([
                np.isclose(row["tfw_x_scale"], row["geotiff_x_scale"]),
                np.isclose(row["tfw_y_scale"], row["geotiff_y_scale"]),
                np.isclose(row["tfw_x_origin"], row["geotiff_x_origin"]),
                np.isclose(row["tfw_y_origin"], row["geotiff_y_origin"]),
            ]):
                matches += 1

        print(f"Files with matching parameters: {matches}/{total}")

    # Report files with errors
    error_mask = gdf["errors"].apply(lambda x: len(x) > 0)
    if error_mask.any():
        print(f"\nFiles with processing notes: {error_mask.sum()}")

# Create the spatial index
gdf_quadrillage_geotiff = create_geotiff_index(GEOTIFF_PATH, OUTPUT_QUADRILLAGE_PARQUET_PATH)

# Analyze results
analyze_tfw_geotiff(gdf_quadrillage_geotiff)


### Coverage Analysis

In [ ]:
def clean_filename(filename):
    """
    Extract tile number from filename.
    Example: '24851110.tif' -> '24851110'
    """
    return filename.replace(".tif", "")

def visualize_coverage(gdf, output_path):
    """
    Create comprehensive visualizations of orthophoto coverage.
    
    Generates multiple views to assess data completeness and quality:
    1. Overall coverage map
    2. Tile grid with identifiers
    3. File size distribution heatmap
    
    Parameters:
        gdf (GeoDataFrame): Spatial index of orthophotos
        output_path (str): Directory for output visualizations
        
    Returns:
        dict: Coverage statistics
    """
    output_path = Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)

    # Plot 1: Coverage Overview
    fig, ax = plt.subplots(figsize=(15, 15))

    # Define Swiss national extent
    swiss_bounds = box(2485000, 1075000, 2834000, 1299000)
    x, y = swiss_bounds.exterior.xy
    ax.plot(x, y, "r--", label="Swiss National Extent", linewidth=2)

    # Plot all tile footprints
    gdf.plot(ax=ax, alpha=0.5, edgecolor="blue", facecolor="none")
    ax.set_title("Orthophoto Coverage Overview", fontsize=16)
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.savefig(output_path / "01_coverage_overview.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Plot 2: Detailed Tile Grid
    fig, ax = plt.subplots(figsize=(15, 15))

    # Color-code tiles by latitude for visual organization
    y_coords = np.array([geom.bounds[1] for geom in gdf.geometry])
    norm = Normalize(vmin=y_coords.min(), vmax=y_coords.max())

    # Plot each tile with its identifier
    for idx, row in gdf.iterrows():
        color = plt.cm.viridis(norm(y_coords[idx]))
        x, y = row.geometry.exterior.xy
        ax.fill(x, y, alpha=0.5, facecolor=color, edgecolor="black", linewidth=0.5)

        # Add tile number at centroid
        centroid = row.geometry.centroid
        ax.annotate(
            clean_filename(row.file_name),
            (centroid.x, centroid.y),
            fontsize=3,
            ha="center",
            va="center",
            color="black",
            weight="light",
        )

    ax.set_title("Tile Grid with Identifiers", fontsize=16)
    ax.grid(True, alpha=0.3)

    # Add colorbar for North-South gradient
    sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
    plt.colorbar(sm, ax=ax, label="Y-coordinate (North-South)")

    plt.savefig(output_path / "02_tile_grid.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Plot 3: File Size Distribution
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create scatter plot colored by file size
    scatter = ax.scatter(
        [geom.centroid.x for geom in gdf.geometry],
        [geom.centroid.y for geom in gdf.geometry],
        c=gdf["size_mb"],
        s=100,
        alpha=0.6,
        cmap="YlOrRd",
    )

    plt.colorbar(scatter, ax=ax, label="File Size (MB)")
    ax.set_title("File Size Distribution Across Coverage Area", fontsize=16)
    ax.grid(True, alpha=0.3)
    plt.savefig(output_path / "03_size_distribution.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Calculate comprehensive statistics
    bounds = gdf.total_bounds
    x_coords = sorted(set([int(geom.bounds[0]) for geom in gdf.geometry]))
    y_coords = sorted(set([int(geom.bounds[1]) for geom in gdf.geometry]))

    # Determine tile dimensions
    tile_width = abs(gdf.iloc[0].geometry.bounds[2] - gdf.iloc[0].geometry.bounds[0])
    tile_height = abs(gdf.iloc[0].geometry.bounds[3] - gdf.iloc[0].geometry.bounds[1])

    stats = {
        "total_area_km2": gdf.geometry.area.sum() / 1_000_000,
        "tile_count": len(gdf),
        "x_min": bounds[0],
        "y_min": bounds[1],
        "x_max": bounds[2],
        "y_max": bounds[3],
        "width_km": (bounds[2] - bounds[0]) / 1000,
        "height_km": (bounds[3] - bounds[1]) / 1000,
        "grid_width": len(x_coords),
        "grid_height": len(y_coords),
        "tile_width_m": tile_width,
        "tile_height_m": tile_height,
        "total_size_gb": gdf["size_mb"].sum() / 1024,
        "mean_size_mb": gdf["size_mb"].mean(),
        "min_size_mb": gdf["size_mb"].min(),
        "max_size_mb": gdf["size_mb"].max(),
    }

    # Generate and save comprehensive statistics report
    with open(output_path / "04_coverage_statistics.txt", "w") as f:
        print("\n=== COVERAGE STATISTICS ===", file=f)
        print("\nSpatial Coverage:", file=f)
        print(f"  Total area: {stats['total_area_km2']:.2f} km²", file=f)
        print(f"  Number of tiles: {stats['tile_count']}", file=f)
        print("\nGeographic Extent:", file=f)
        print(f"  X range: {stats['x_min']:.2f} to {stats['x_max']:.2f}", file=f)
        print(f"  Y range: {stats['y_min']:.2f} to {stats['y_max']:.2f}", file=f)
        print(f"  Width: {stats['width_km']:.2f} km", file=f)
        print(f"  Height: {stats['height_km']:.2f} km", file=f)
        print("\nGrid Structure:", file=f)
        print(f"  Grid dimensions: {stats['grid_width']} × {stats['grid_height']} tiles", file=f)
        print(f"  Tile size: {stats['tile_width_m']:.2f}m × {stats['tile_height_m']:.2f}m", file=f)
        print("\nData Volume:", file=f)
        print(f"  Total size: {stats['total_size_gb']:.2f} GB", file=f)
        print(f"  Average file size: {stats['mean_size_mb']:.2f} MB", file=f)
        print(f"  Size range: {stats['min_size_mb']:.2f} - {stats['max_size_mb']:.2f} MB", file=f)

    # Display statistics in console
    with open(output_path / "04_coverage_statistics.txt", "r") as f:
        print(f.read())

    return stats

# Generate coverage analysis
stats = visualize_coverage(gdf_quadrillage_geotiff, VIZ_OUTPUT_PATH)

### Data Quality Checks

In [ ]:
def check_duplicates(gdf):
    """
    Comprehensive duplicate detection and data quality analysis.
    
    Identifies various types of duplicates and inconsistencies:
    - File system duplicates
    - Coordinate duplicates
    - Geometry overlaps
    - Metadata inconsistencies
    
    Parameters:
        gdf (GeoDataFrame): Spatial index to analyze
    """
    print("\n=== COMPREHENSIVE DATA QUALITY ANALYSIS ===")
    print(f"\nAnalyzing {len(gdf)} files...\n")

    print("1. FILE SYSTEM INTEGRITY")
    print("-" * 50)
    
    # Check for duplicate file paths
    print("Checking for duplicate file references...")
    file_dups = gdf[gdf['file_path'].duplicated(keep='first')]
    if len(file_dups) > 0:
        print(f"WARNING: Found {len(file_dups)} duplicate file paths")
        for idx, row in file_dups.iterrows():
            print(f"  - {row['file_path']}")
    else:
        print("✓ No duplicate file paths found")

    # Check for duplicate filenames
    print("\nChecking for duplicate filenames...")
    name_dups = gdf[gdf['file_name'].duplicated(keep='first')]
    if len(name_dups) > 0:
        print(f"WARNING: Found {len(name_dups)} duplicate filenames")
        for idx, row in name_dups.iterrows():
            print(f"  - {row['file_name']} at {row['file_path']}")
    else:
        print("✓ No duplicate filenames found")

    print("\n2. SPATIAL INTEGRITY")
    print("-" * 50)
    
    # Check TFW coordinate duplicates
    print("Checking for duplicate TFW coordinates...")
    tfw_dups = gdf[gdf.duplicated(
        subset=['tfw_x_origin', 'tfw_y_origin'], 
        keep='first'
    )]
    if len(tfw_dups) > 0:
        print(f"WARNING: Found {len(tfw_dups)} files with duplicate TFW origins")
        for idx, row in tfw_dups.iterrows():
            print(f"  - {row['file_name']} at ({row['tfw_x_origin']}, {row['tfw_y_origin']})")
    else:
        print("✓ No duplicate TFW origins found")

    # Check GeoTIFF coordinate duplicates
    print("\nChecking for duplicate GeoTIFF coordinates...")
    geotiff_dups = gdf[gdf.duplicated(
        subset=['geotiff_x_origin', 'geotiff_y_origin'], 
        keep='first'
    )]
    if len(geotiff_dups) > 0:
        print(f"WARNING: Found {len(geotiff_dups)} files with duplicate GeoTIFF origins")
    else:
        print("✓ No duplicate GeoTIFF origins found")

    print("\n3. GEOMETRY ANALYSIS")
    print("-" * 50)
    
    # Check for exact geometry duplicates
    print("Checking for identical geometries...")
    geom_dups = gdf[gdf.geometry.apply(lambda x: x.wkb).duplicated(keep='first')]
    if len(geom_dups) > 0:
        print(f"WARNING: Found {len(geom_dups)} duplicate geometries")
    else:
        print("✓ No duplicate geometries found")

    # Check for overlapping tiles
    print("\nAnalyzing tile overlaps...")
    overlaps = []
    
    # Use spatial index for efficient overlap detection
    for idx1, row1 in gdf.iterrows():
        possible_matches_idx = list(gdf.sindex.intersection(row1.geometry.bounds))
        possible_matches = gdf.iloc[possible_matches_idx]

        for idx2, row2 in possible_matches.iterrows():
            if idx1 < idx2:  # Check each pair only once
                if row1.geometry.intersects(row2.geometry):
                    intersection = row1.geometry.intersection(row2.geometry)
                    if intersection.area > 0:
                        overlap_pct = (intersection.area / row1.geometry.area) * 100
                        if overlap_pct > 1:  # Report significant overlaps only
                            overlaps.append({
                                'file1': row1['file_name'],
                                'file2': row2['file_name'],
                                'overlap_area_m2': intersection.area,
                                'overlap_percentage': overlap_pct,
                                'bounds': intersection.bounds
                            })

    if overlaps:
        print(f"WARNING: Found {len(overlaps)} overlapping tile pairs (>1% overlap)")
        # Show most significant overlaps
        overlaps.sort(key=lambda x: x['overlap_percentage'], reverse=True)
        print("\nTop overlaps:")
        for overlap in overlaps[:5]:
            print(f"  - {overlap['file1']} × {overlap['file2']}: {overlap['overlap_percentage']:.2f}%")
        if len(overlaps) > 5:
            print(f"  ... and {len(overlaps) - 5} more overlaps")
    else:
        print("✓ No significant overlaps found")

    print("\n4. METADATA CONSISTENCY")
    print("-" * 50)
    
    # Check TFW vs GeoTIFF consistency
    print("Checking TFW vs GeoTIFF parameter consistency...")
    has_both = gdf['has_tfw'] & gdf['has_crs']
    files_with_both = gdf[has_both]

    if len(files_with_both) == 0:
        print("No files have both TFW and GeoTIFF metadata for comparison")
    else:
        # Check for parameter mismatches
        inconsistencies = files_with_both[
            (files_with_both['tfw_x_scale'] != files_with_both['geotiff_x_scale']) |
            (files_with_both['tfw_y_scale'] != files_with_both['geotiff_y_scale']) |
            (files_with_both['tfw_x_origin'] != files_with_both['geotiff_x_origin']) |
            (files_with_both['tfw_y_origin'] != files_with_both['geotiff_y_origin'])
        ]

        if len(inconsistencies) > 0:
            print(f"WARNING: Found {len(inconsistencies)} files with parameter inconsistencies")
            # Show first few examples
            for idx, row in inconsistencies.head(3).iterrows():
                print(f"\n  File: {row['file_name']}")
                if row['tfw_x_scale'] != row['geotiff_x_scale']:
                    print(f"    X scale: TFW={row['tfw_x_scale']}, GeoTIFF={row['geotiff_x_scale']}")
                if row['tfw_y_scale'] != row['geotiff_y_scale']:
                    print(f"    Y scale: TFW={row['tfw_y_scale']}, GeoTIFF={row['geotiff_y_scale']}")
        else:
            print("✓ All TFW and GeoTIFF parameters are consistent")

    print("\n=== ANALYSIS COMPLETE ===\n")

# Execute comprehensive quality checks
check_duplicates(gdf_quadrillage_geotiff)